# <center>Data Mining Project 3 Spring semester 2019-2020</center>
## <center>Γεώργιος Μαραγκοζάκης &emsp; 1115201500089</center>

___

### Do all the necessary imports for this notebook

In [1]:
# data processing
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords as nltkStopwords
from string import punctuation, digits
import re
from nltk import word_tokenize
from nltk.stem import PorterStemmer

# visualization
from wordcloud import WordCloud
from IPython.display import Image
from IPython.display import display
from itertools import cycle
import matplotlib.patches as mpatches

# classification
from sklearn.model_selection import KFold, cross_validate
from sklearn import svm, preprocessing
from sklearn.metrics import classification_report, make_scorer, accuracy_score, \
                            precision_score, recall_score, f1_score, roc_curve, auc,\
                            roc_auc_score, plot_roc_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import label_binarize
import scipy
from collections import Counter
import gensim 
import random
from operator import add

# vectorization
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# clustering
from nltk.cluster import KMeansClusterer, cosine_distance
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

# for data exploration
import os
import numpy as np

## __Dataset Preprocessing__

- ### *Read all .csv files*

In [2]:
specificPath = './data/'
# find the column's names of each csv
for fileName in os.listdir(specificPath):
    # we need to check only .csv files
    if fileName.endswith(".csv"):
        thisCsv = pd.read_csv(os.path.join(specificPath, fileName), dtype='unicode')
        if fileName == 'train.csv':
            trainDF = thisCsv
        elif fileName == 'impermium_verification_labels.csv':
            testLabelsDf = thisCsv
        elif fileName == 'impermium_verification_set.csv':
            testSetDf = thisCsv
        else:
            print("Wrong File!")

Wrong File!


__train.csv__

In [3]:
trainDF

,Insult,Date,Comment
0,1,20120618192155Z,"""You fuck your dad."""
1,0,20120528192215Z,"""i really don't understand your point.\xa0 It ..."
2,0,NaN,"""A\\xc2\\xa0majority of Canadians can and has ..."
3,0,NaN,"""listen if you dont wanna get married to a man..."
4,0,20120619094753Z,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd..."
...,...,...,...
3942,1,20120502172717Z,"""you are both morons and that is never happening"""
3943,0,20120528164814Z,"""Many toolbars include spell check, like Yahoo..."
3944,0,20120620142813Z,"""@LambeauOrWrigley\xa0\xa0@K.Moss\xa0\nSioux F..."
3945,0,20120528205648Z,"""How about Felix? He is sure turning into one ..."


__test.csv__

In [4]:
testLabelsDf

,id,Insult,Date,Comment,Usage
0,1,0,20120603163526Z,"""like this if you are a tribe fan""",PrivateTest
1,2,1,20120531215447Z,"""you're idiot.......................""",PrivateTest
2,3,1,20120823164228Z,"""I am a woman Babs, and the only ""war on women...",PrivateTest
3,4,1,20120826010752Z,"""WOW & YOU BENEFITTED SO MANY WINS THIS YEAR F...",PrivateTest
4,5,1,20120602223825Z,"""haha green me red you now loser whos winning ...",PrivateTest
...,...,...,...,...,...
2230,2231,0,20120528100303Z,"""FUCKIN LAME COME ON WTF STOP FUCKING OVER MY ...",PrivateTest
2231,2232,1,20120531185813Z,"""YOU SHUT YOUR IGNORANT PIE HOLE YOU LITTLE IN...",PrivateTest
2232,2233,0,20120529130822Z,"""sweetie pie is looking very much like her cou...",PrivateTest
2233,2234,1,20120531045826Z,"""ball4real where are you with your miami g-ayn...",PrivateTest


In [5]:
testSetDf
# - ### *Convert all comments to lower case*

,id,Insult,Date,Comment,Usage
0,1,NaN,20120603163526Z,"""like this if you are a tribe fan""",PrivateTest
1,2,NaN,20120531215447Z,"""you're idiot.......................""",PrivateTest
2,3,NaN,20120823164228Z,"""I am a woman Babs, and the only ""war on women...",PrivateTest
3,4,NaN,20120826010752Z,"""WOW & YOU BENEFITTED SO MANY WINS THIS YEAR F...",PrivateTest
4,5,NaN,20120602223825Z,"""haha green me red you now loser whos winning ...",PrivateTest
...,...,...,...,...,...
2230,2231,NaN,20120528100303Z,"""FUCKIN LAME COME ON WTF STOP FUCKING OVER MY ...",PrivateTest
2231,2232,NaN,20120531185813Z,"""YOU SHUT YOUR IGNORANT PIE HOLE YOU LITTLE IN...",PrivateTest
2232,2233,NaN,20120529130822Z,"""sweetie pie is looking very much like her cou...",PrivateTest
2233,2234,NaN,20120531045826Z,"""ball4real where are you with your miami g-ayn...",PrivateTest


In [6]:
trainDF['Comment'] = trainDF['Comment'].str.lower()

trainDF

,Insult,Date,Comment
0,1,20120618192155Z,"""you fuck your dad."""
1,0,20120528192215Z,"""i really don't understand your point.\xa0 it ..."
2,0,NaN,"""a\\xc2\\xa0majority of canadians can and has ..."
3,0,NaN,"""listen if you dont wanna get married to a man..."
4,0,20120619094753Z,"""c\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd..."
...,...,...,...
3942,1,20120502172717Z,"""you are both morons and that is never happening"""
3943,0,20120528164814Z,"""many toolbars include spell check, like yahoo..."
3944,0,20120620142813Z,"""@lambeauorwrigley\xa0\xa0@k.moss\xa0\nsioux f..."
3945,0,20120528205648Z,"""how about felix? he is sure turning into one ..."


In [7]:
testLabelsDf['Comment'] = testLabelsDf['Comment'].str.lower()

testLabelsDf

,id,Insult,Date,Comment,Usage
0,1,0,20120603163526Z,"""like this if you are a tribe fan""",PrivateTest
1,2,1,20120531215447Z,"""you're idiot.......................""",PrivateTest
2,3,1,20120823164228Z,"""i am a woman babs, and the only ""war on women...",PrivateTest
3,4,1,20120826010752Z,"""wow & you benefitted so many wins this year f...",PrivateTest
4,5,1,20120602223825Z,"""haha green me red you now loser whos winning ...",PrivateTest
...,...,...,...,...,...
2230,2231,0,20120528100303Z,"""fuckin lame come on wtf stop fucking over my ...",PrivateTest
2231,2232,1,20120531185813Z,"""you shut your ignorant pie hole you little in...",PrivateTest
2232,2233,0,20120529130822Z,"""sweetie pie is looking very much like her cou...",PrivateTest
2233,2234,1,20120531045826Z,"""ball4real where are you with your miami g-ayn...",PrivateTest


In [8]:
testSetDf['Comment'] = testSetDf['Comment'].str.lower()

testSetDf

,id,Insult,Date,Comment,Usage
0,1,NaN,20120603163526Z,"""like this if you are a tribe fan""",PrivateTest
1,2,NaN,20120531215447Z,"""you're idiot.......................""",PrivateTest
2,3,NaN,20120823164228Z,"""i am a woman babs, and the only ""war on women...",PrivateTest
3,4,NaN,20120826010752Z,"""wow & you benefitted so many wins this year f...",PrivateTest
4,5,NaN,20120602223825Z,"""haha green me red you now loser whos winning ...",PrivateTest
...,...,...,...,...,...
2230,2231,NaN,20120528100303Z,"""fuckin lame come on wtf stop fucking over my ...",PrivateTest
2231,2232,NaN,20120531185813Z,"""you shut your ignorant pie hole you little in...",PrivateTest
2232,2233,NaN,20120529130822Z,"""sweetie pie is looking very much like her cou...",PrivateTest
2233,2234,NaN,20120531045826Z,"""ball4real where are you with your miami g-ayn...",PrivateTest


In [9]:
# - ### *Delete all punctuation*
# url detection from https://www.w3resource.com/python-exercises/re/python-re-exercise-42.php

In [10]:
trainDF['Comment'] = trainDF['Comment'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',' ', regex=True)
#trainDF['Comment'] = trainDF['Comment'].str.replace("\\[a-z]|[\]", " ", regex=True)

trainDF['Comment'] = trainDF['Comment'].str.replace(r"\\+\w+", "", regex=True)

#trainDF['Comment'] = trainDF['Comment'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
#trainDF['Comment'] = trainDF['Comment'].apply(lambda x: re.split('http:\/\/.*', str(x))[0])

# trainDF['Comment'] = trainDF['Comment'].replace('https:.^\s\n\r','', regex=True)
# trainDF['Comment'] = trainDF['Comment'].replace('http:.^\s\n\r','', regex=True)
#trainDF["Comment"] = trainDF["Comment"].str.replace('[^\w\s]',' ')
#trainDF["Comment"] = trainDF["Comment"].str.replace('[^\w\s]','', regex=True)
#trainDF['Comment'] = trainDF['Comment'].str.replace('[/.,\/#!$%\^&\*;:=\-_`~/g]', ' ', regex=True)
#trainDF['Comment'] = trainDF['Comment'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
trainDF.to_csv('./data/deletedTrain.csv')

trainDF

,Insult,Date,Comment
0,1,20120618192155Z,"""you fuck your dad."""
1,0,20120528192215Z,"""i really don't understand your point. it seem..."
2,0,NaN,"""a of canadians can and has been wrong before ..."
3,0,NaN,"""listen if you dont wanna get married to a man..."
4,0,20120619094753Z,"""c b xu bi t 2011 c ho kh ? ng d ng cu chi..."
...,...,...,...
3942,1,20120502172717Z,"""you are both morons and that is never happening"""
3943,0,20120528164814Z,"""many toolbars include spell check, like yahoo..."
3944,0,20120620142813Z,"""@lambeauorwrigley@k.moss falls, s.d. i told m..."
3945,0,20120528205648Z,"""how about felix? he is sure turning into one ..."


In [11]:
testLabelsDf["Comment"] = testLabelsDf["Comment"].str.replace('[^\w\s]','')

testLabelsDf

,id,Insult,Date,Comment,Usage
0,1,0,20120603163526Z,like this if you are a tribe fan,PrivateTest
1,2,1,20120531215447Z,youre idiot,PrivateTest
2,3,1,20120823164228Z,i am a woman babs and the only war on women i ...,PrivateTest
3,4,1,20120826010752Z,wow you benefitted so many wins this year fro...,PrivateTest
4,5,1,20120602223825Z,haha green me red you now loser whos winning n...,PrivateTest
...,...,...,...,...,...
2230,2231,0,20120528100303Z,fuckin lame come on wtf stop fucking over my b...,PrivateTest
2231,2232,1,20120531185813Z,you shut your ignorant pie hole you little ins...,PrivateTest
2232,2233,0,20120529130822Z,sweetie pie is looking very much like her cous...,PrivateTest
2233,2234,1,20120531045826Z,ball4real where are you with your miami gayness,PrivateTest


In [12]:
testSetDf["Comment"] = testSetDf["Comment"].str.replace('[^\w\s]','')

testSetDf

,id,Insult,Date,Comment,Usage
0,1,NaN,20120603163526Z,like this if you are a tribe fan,PrivateTest
1,2,NaN,20120531215447Z,youre idiot,PrivateTest
2,3,NaN,20120823164228Z,i am a woman babs and the only war on women i ...,PrivateTest
3,4,NaN,20120826010752Z,wow you benefitted so many wins this year fro...,PrivateTest
4,5,NaN,20120602223825Z,haha green me red you now loser whos winning n...,PrivateTest
...,...,...,...,...,...
2230,2231,NaN,20120528100303Z,fuckin lame come on wtf stop fucking over my b...,PrivateTest
2231,2232,NaN,20120531185813Z,you shut your ignorant pie hole you little ins...,PrivateTest
2232,2233,NaN,20120529130822Z,sweetie pie is looking very much like her cous...,PrivateTest
2233,2234,NaN,20120531045826Z,ball4real where are you with your miami gayness,PrivateTest
